In [1]:
import urllib.request
import api_client
import tqdm
import time
import numpy as np
from numpy import array
from matplotlib import pyplot as plt

In [2]:
a = api_client.APIClient('http://localhost:5000')
a

APIClient(connected on http://localhost:5000)

In [3]:
def set_string(in_str):
    url = f'http://10.156.2.28:8090/{in_str}'
    res = urllib.request.urlopen(url).read().decode()

In [4]:
from continuous_calibration import *
from second_order_results import * 

In [5]:
#s1 = tif_to_signal('053123_test/0_0.tif')
#s2 = tif_to_signal('0_0_3pm.tif')
#s2 = tif_to_signal('0_0.tif')
#s2 = tif_to_signal('053123_test/160_0.tif')

In [6]:
#cs_off,off = calculate_signal_offset(s1, s2)

In [7]:
#fit_calibration, fit_hamamatsu = get_calibrations()

In [8]:
#plt.plot( 
#    #[cs_off(fit_calibration[0](fit_hamamatsu[0].invert(i)[0])) for i in range(2304)]
#    map_calibration(cs_off, fit_calibration[0], fit_hamamatsu[0])
#)

In [9]:
correction_test = {0: array([ 3.73212399e-30, -4.69891869e-26,  2.58843347e-22, -8.12279970e-19,
         1.58321529e-15, -1.96093350e-12,  1.51860394e-09, -6.96725472e-07,
         1.70168791e-04, -1.78011831e-02,  2.04031691e-01]),
 1: array([-1.28736653e-28,  1.44153132e-24, -6.82309114e-21,  1.77782496e-17,
        -2.78381873e-14,  2.68739513e-11, -1.58308711e-08,  5.47177894e-06,
        -1.03005531e-03,  8.28313372e-02,  3.33969277e+00]),
 2: array([ 9.05016552e-30, -1.12305115e-25,  5.96962476e-22, -1.77396475e-18,
         3.22382363e-15, -3.67753890e-12,  2.59278964e-09, -1.06036088e-06,
         2.14478933e-04, -1.27329473e-02, -9.95703084e-02])}


In [64]:
a.apply_dac_wavetable(
    0,
    calculate_calibrated_galvo(
        0,
        second_order_correction=lambda x: np.polyval(
            correction_test[0], x
        )
        #time_offset=map_calibration(cs_off, fit_calibration[0], fit_hamamatsu[0])
    )
)
a.apply_dac_wavetable(
    1,
    calculate_calibrated_galvo(
        2,
        second_order_correction=lambda x: np.polyval(
            correction_test[2], x
        ),
    )
)
a.apply_dac_wavetable(
    2,
    calculate_calibrated_galvo(
        1,
        second_order_correction=lambda x: np.polyval(
            correction_test[1], x
        ),
        second_scale=-1.0
    )
)

a.reset_galvo(0)
a.reset_galvo(1)
a.reset_galvo(2)

'Done.'

set_string('2nd_cal_4_test2')
for i in range(3):
    a.trigger_expanded('A', 10, True, True)

In [13]:
#a.trigger_expanded('A', 4000, False, False)

In [14]:
a.location()

{'1': 0.0, '2': -8.12, '3': 0.0}

In [15]:
#a.move(x=-26)

In [16]:
set_string("init_testing")

In [17]:
#for dz in range(5,10,5):
#    for dx in range(-20,40,20):
#while True:
#        for dy in range(-20,40,20):
#            a.velocity(x=20,y=20,z=.3)
#            a.move(x=dx, y=dy, z=dz)
#            a.wait_for_move()

In [18]:
a.is_moving()

False

In [19]:
#while True: 
#    set_string(f'{time.time()}')
#    a.trigger_expanded('A', 1, False, False) # True, True)
#    time.sleep(0.5)

In [20]:
#set_string(f'{time.time()}')
#a.trigger_expanded('A', 2000, True, True)

In [21]:
a.location()

{'1': 0.0, '2': -8.12, '3': 0.0}

In [51]:
#axis1 = (-21.2, -7.6)
axis1_start = -15.0
axis2_start = 21.1

STEP_SIZE = 0.21

axis1 = (axis1_start-.3,axis1_start+.3)
axis2 = (axis2_start-.3,axis2_start+.3)

In [52]:
axis_steps = []
STEP_SIZE = 0.21
current1 = axis1[0]
while current1 < axis1[1]:
    current2 = axis2[0]
    while current2 < axis2[1]:
        axis_steps.append(
            (
                round(current1, 1),
                round(current2, 1)
            )
        )
        current2 += STEP_SIZE
    current1 += STEP_SIZE

In [53]:
axis_steps = []

X, Y = 20, 20
x, y, dx, dy = 0, 0, 0, -1
for i in range(max(X,Y)**2):
    if(-X/2 < x <= X/2) and (-Y/2 < y <= Y/2):
        axis_steps.append(
            (
                round(axis1_start+(x*STEP_SIZE), 3),
                round(axis2_start+(y*STEP_SIZE), 3)
            )
        )
    if x==y or (x < 0 and x==-y) or (x > 0 and x==1-y):
        dx, dy = -dy, dx
    x, y = x+dx, y+dy

### high z

In [54]:
print( len(axis_steps) )
axis_steps[:10]

400


[(-15.0, 21.1),
 (-14.79, 21.1),
 (-14.79, 21.31),
 (-15.0, 21.31),
 (-15.21, 21.31),
 (-15.21, 21.1),
 (-15.21, 20.89),
 (-15.0, 20.89),
 (-14.79, 20.89),
 (-14.58, 20.89)]

In [56]:
a.velocity(x=.1,y=.1)
a.move(x=axis_steps[0][0], y=axis_steps[0][1])

In [41]:
axis_steps[2]

(-15.49, 14.81)

In [57]:
a.location()

{'1': 19.659, '2': -14.999, '3': 21.1}

In [65]:
a.trigger_expanded('A', 1, True, True)

'Done.'

In [62]:
axis_steps[1]

(-14.79, 21.1)

In [66]:
a.velocity(x=1, y=1)
for current1, current2 in tqdm.tqdm(axis_steps):
    for _ in range(2):
        a.move(x=current1, y=current2)
    scan_name = f'coord_{current1:+.3f},{current2:+.3f}'
    print(scan_name)
    set_string( scan_name )
    a.wait_for_move()
    time.sleep(1)
    a.trigger_expanded('A', 2000, True, True)
    time.sleep(10)

  0%|                                                   | 0/400 [00:00<?, ?it/s]

coord_-15.000,+21.100


  0%|                                         | 1/400 [00:53<5:54:13, 53.27s/it]

coord_-14.790,+21.100


  0%|▏                                        | 2/400 [01:46<5:53:26, 53.28s/it]

...

 73%|███████████████████████████          | 292/400 [4:19:51<1:36:13, 53.46s/it]

coord_-13.110,+20.050


 73%|███████████████████████████          | 293/400 [4:20:44<1:35:17, 53.44s/it]

coord_-13.110,+20.260


 73%|███████████████████████████          | 293/400 [4:21:26<1:35:28, 53.54s/it]

KeyboardInterrupt

